# Note Or Chord Detect Classifier

First install the libraries needed to train the model.

In [19]:
!pip install sklearn keras librosa pandas matplotlib numpy tensorflow

     |████████████████████████████████| 120.8 MB 13.2 MB/s eta 0:00:01    |█████████████▎                  | 50.0 MB 24.8 MB/s eta 0:00:03     |█████████████████████████▋      | 96.7 MB 20.7 MB/s eta 0:00:02     |███████████████████████████▊    | 104.8 MB 20.7 MB/s eta 0:00:01     |█████████████████████████████▏  | 110.1 MB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 35.1 MB/s eta 0:00:01
Processing /Users/shounakamladi/Library/Caches/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6/termcolor-1.1.0-cp37-none-any.whl
     |████████████████████████████████| 3.8 MB 46.6 MB/s eta 0:00:01
  Using cached google_pasta-0.1.8-py3-none-any.whl (57 kB)
     |████████████████████████████████| 2.5 MB 1.8 MB/s eta 0:00:01     |██▋                             | 204 kB 1.8 MB/s eta 0:00:02     |█████████▍                      | 727 kB 1.8 MB/s eta 0:00:02
Processing /Users/shounakamladi/Library/Caches/pip/wheels/2c/b1/94/43d03e130b929aae7ba3f8d15cbd7bc0d1

Import the modules required.

In [11]:
import math
import sys
import wave
import random
from itertools import islice, zip_longest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import librosa
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils, to_categorical
from keras import regularizers
from sklearn import metrics 

### Generating the Trainning Data

Using a sin waveform and PyAudio, generate various major/minor chord waves,and note waves. I gathered about 600 total, 200 each sample. 

In [12]:
import math
import wave
import struct


def synthComplex(freq=[440],coef=[1], datasize=10000, fname="test.wav"):
    frate = 44100.00  
    amp=8000.0 
    sine_list=[]
    for x in range(datasize):
        samp = 0
        for k in range(len(freq)):
            samp = samp + coef[k] * math.sin(2*math.pi*freq[k]*(x/frate))
        sine_list.append(samp)
    wav_file=wave.open(fname,"w")
    nchannels = 1
    sampwidth = 2
    framerate = int(frate)
    nframes=datasize
    comptype= "NONE"
    compname= "not compressed"
    wav_file.setparams((nchannels, sampwidth, framerate, nframes, comptype, compname))
    for s in sine_list:
        wav_file.writeframes(struct.pack('h', int(s*amp/2)))
    wav_file.close()

# synthComplex([261], [1], 30000, "tone.wav")

These functions acutally generate the wave files of a chord or note for the dataset. Each file is a little less than a second long.

In [4]:
def genChordRoot():
    n = random.randint(15,8000)
    return n
#funcitons finds nth half step pitch
def getNHalfStepPitch(root, halfsteps):
    return root * (2**(1/12))**halfsteps
def generateMajorChordDataset(samples=10, directory=""):
    for i in range(samples):
        root = genChordRoot()
        synthComplex([root,getNHalfStepPitch(root, 4),getNHalfStepPitch(root,7)],[1,1,1], 30000, directory+"/MajorChord_"+str(i)+".wav")
def generateMinorChordDataset(samples=10, directory=""):
    for i in range(samples):
        root = genChordRoot()
        synthComplex([root,getNHalfStepPitch(root, 3),getNHalfStepPitch(root,7)],[1,1,1], 30000, directory+"/MinorChord_"+str(i)+".wav")

def generateNotesDataset(samples=10, directory=""):
    for i in range(samples):
        root = genChordRoot()
        synthComplex([root],[1], 30000, directory+"/Note_"+str(i)+".wav")
        


In [5]:
# generateMinorChordDataset(200,"minor")
# generateMajorChordDataset(200,"major")
# generateNotesDataset(200,"notes")

In [6]:
dataset = pd.DataFrame(columns=["soundfile", "class"])
#notes
noteName = "notes/Note_"
for sample in range(200):
    dataset.loc[sample,:]= {"soundfile":[noteName+str(sample)+".wav"], "class":["note"]}
# major chord
chordName = "major/MajorChord_"
for sample in range(200,400):
    dataset.loc[sample,:]= {"soundfile":[chordName+str(sample%200)+".wav"], "class":["major"]}
    
# minor chord
chordName = "minor/MinorChord_"
for sample in range(400,600):
    dataset.loc[sample,:]= {"soundfile":[chordName+str(sample%200)+".wav"], "class":["minor"]}

In [11]:
file = "out.csv"
f = open(file, 'w')
f.write("soundfile, class\n")
for sample in range(len(dataset)):
    str1 = dataset.loc[sample,"soundfile"][0]
    str2 = dataset.loc[sample,"class"][0]
    row = str1+","+str2
    f.write(row+"\n")
f.close()

In [13]:
metadata = pd.read_csv('out.csv')
features=[]
max_pad_len = 174
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

for index, row in metadata.iterrows():
    
    file_name = str(row["soundfile"])
    
    class_label = str(row[1])
    data = extract_features(file_name)
    
    features.append([data, class_label])
    
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
print('Finished feature extraction from ', len(featuresdf), ' files')

Finished feature extraction from  600  files


In [32]:
# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [33]:
num_rows = 40
num_columns = 174
num_channels = 1

X_train = X_train.reshape(X_train.shape[0], num_rows, num_columns, num_channels)
X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(32, kernel_regularizer=regularizers.l2(0.25)))
model.add(Dense(num_labels, activation='softmax'))

In [34]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(X_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 39, 173, 16)       80        
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 19, 86, 16)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 19, 86, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 18, 85, 32)        2080      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 9, 42, 32)         0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 9, 42, 32)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 8, 41, 64)        

In [35]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

#num_epochs = 12
#num_batch_size = 128

num_epochs = 81
num_batch_size = 256

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 480 samples, validate on 120 samples
Epoch 1/81
480/480 [==============================] - 2s 4ms/step - loss: 27.5048 - accuracy: 0.3125 - val_loss: 24.3520 - val_accuracy: 0.3500

Epoch 00001: val_loss improved from inf to 24.35204, saving model to saved_models/weights.best.basic_cnn_2.hdf5
Epoch 2/81
480/480 [==============================] - 2s 3ms/step - loss: 26.6952 - accuracy: 0.3479 - val_loss: 23.3697 - val_accuracy: 0.3500

Epoch 00002: val_loss improved from 24.35204 to 23.36967, saving model to saved_models/weights.best.basic_cnn_2.hdf5
Epoch 3/81
480/480 [==============================] - 1s 3ms/step - loss: 24.8687 - accuracy: 0.3396 - val_loss: 23.2168 - val_accuracy: 0.3667

Epoch 00003: val_loss improved from 23.36967 to 23.21680, saving model to saved_models/weights.best.basic_cnn_2.hdf5
Epoch 4/81
480/480 [==============================] - 1s 3ms/step - loss: 24.4792 - accuracy: 0.3812 - val_loss: 22.8177 - val_accuracy: 0.3667

Epoch 00004: val_loss improv

480/480 [==============================] - 1s 3ms/step - loss: 12.6852 - accuracy: 0.6562 - val_loss: 12.5025 - val_accuracy: 0.7000

Epoch 00032: val_loss improved from 12.80202 to 12.50252, saving model to saved_models/weights.best.basic_cnn_2.hdf5
Epoch 33/81
480/480 [==============================] - 2s 3ms/step - loss: 12.3747 - accuracy: 0.6313 - val_loss: 12.1964 - val_accuracy: 0.7000

Epoch 00033: val_loss improved from 12.50252 to 12.19640, saving model to saved_models/weights.best.basic_cnn_2.hdf5
Epoch 34/81
480/480 [==============================] - 2s 4ms/step - loss: 12.0945 - accuracy: 0.6604 - val_loss: 11.8945 - val_accuracy: 0.7000

Epoch 00034: val_loss improved from 12.19640 to 11.89450, saving model to saved_models/weights.best.basic_cnn_2.hdf5
Epoch 35/81
480/480 [==============================] - 2s 4ms/step - loss: 11.8097 - accuracy: 0.6646 - val_loss: 11.6096 - val_accuracy: 0.7000

Epoch 00035: val_loss improved from 11.89450 to 11.60962, saving model to sav


Epoch 00063: val_loss improved from 6.13002 to 5.98735, saving model to saved_models/weights.best.basic_cnn_2.hdf5
Epoch 64/81
480/480 [==============================] - 2s 3ms/step - loss: 5.9210 - accuracy: 0.7958 - val_loss: 5.8467 - val_accuracy: 0.7917

Epoch 00064: val_loss improved from 5.98735 to 5.84665, saving model to saved_models/weights.best.basic_cnn_2.hdf5
Epoch 65/81
480/480 [==============================] - 2s 3ms/step - loss: 5.7858 - accuracy: 0.7729 - val_loss: 5.7137 - val_accuracy: 0.7917

Epoch 00065: val_loss improved from 5.84665 to 5.71373, saving model to saved_models/weights.best.basic_cnn_2.hdf5
Epoch 66/81
480/480 [==============================] - 1s 3ms/step - loss: 5.6402 - accuracy: 0.7958 - val_loss: 5.5858 - val_accuracy: 0.8000

Epoch 00066: val_loss improved from 5.71373 to 5.58580, saving model to saved_models/weights.best.basic_cnn_2.hdf5
Epoch 67/81
480/480 [==============================] - 2s 3ms/step - loss: 5.5226 - accuracy: 0.7958 - val_

In [36]:
# Calculate pre-training accuracy 
score = model.evaluate(X_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Trained accuracy: %.4f%%" % accuracy)

120/120 [==============================] - 0s 1ms/step
Trained accuracy: 82.5000%
